In [3]:
import xarray as xr
import geoviews as gv
import holoviews as hv
from holoviews import opts
import panel as pn
from cartopy import crs as ccrs

# Load Panel extension
pn.extension()  # This is required for Panel to work properly in a Jupyter Notebook

# Load the NetCDF file
nc_file_path = "all_gwl_humidex_max_with_dates.nc"
ds = xr.open_dataset(nc_file_path)

# Define GWL-specific year ranges
year_ranges = {
    "1.0": (1995, 2014),
    "1.5": (2008, 2027),
    "2.0": (2018, 2037),
    "2.5": (2029, 2048),
    "3.0": (2037, 2056),
    "4.0": (2052, 2071),
}

# Interactive widgets
gwl_selector = pn.widgets.Select(name="Select GWL", options=list(year_ranges.keys()), value="4.0")
year_selector = pn.widgets.IntSlider(name="Select Year", start=2052, end=2071, step=1, value=2052)

# Function to update the year slider dynamically based on the selected GWL
@pn.depends(gwl_selector.param.value, watch=True)
def update_year_slider(gwl):
    start_year, end_year = year_ranges[gwl]
    year_selector.start = start_year
    year_selector.end = end_year
    year_selector.value = start_year

# Function to load and plot data for the selected GWL and year
@pn.depends(gwl_selector.param.value, year_selector.param.value)
def plot_humidex(gwl, year):
    # Calculate relative year index
    start_year = year_ranges[gwl][0]
    relative_year = year - start_year
    
    # Extract data for the selected GWL and year
    humidex_field = ds["humidex_max"].sel(gwl=gwl, year=relative_year)
    lat_percentile = ds["lat_percentile"].sel(gwl=gwl, year=relative_year).values
    lon_percentile = ds["lon_percentile"].sel(gwl=gwl, year=relative_year).values
    lat_abs_max = ds["lat_abs_max"].sel(gwl=gwl, year=relative_year).values
    lon_abs_max = ds["lon_abs_max"].sel(gwl=gwl, year=relative_year).values
    date = ds["date_of_max_percentile"].sel(gwl=gwl, year=relative_year).values
    absolute_max_value = humidex_field.max().values
    percentile_max_value = float(
        humidex_field.quantile(0.999, skipna=True).values
    )
    
    # Convert humidex_field to a GeoViews dataset
    gv_data = gv.Dataset(
        (ds["longitude"], ds["latitude"], humidex_field),
        kdims=["Longitude", "Latitude"],
        vdims=["Humidex"],
        crs=ccrs.PlateCarree(),
    )
    
    # Create QuadMesh plot
    quadmesh = gv_data.to(gv.QuadMesh, ["Longitude", "Latitude"], ["Humidex"]).opts(
        cmap="RdBu_r",
        colorbar=True,
        clim=(20, 50),  # Fixed color bar levels
        tools=["hover"],
        projection=ccrs.PlateCarree(),
        width=800,
        height=600,
        title=f"GWL {gwl}, Date: {date}",
    )
    
    # Add markers for 99.9th percentile and absolute max
    markers = gv.Points(
        [(lon_percentile, lat_percentile, "99.9th Percentile", "cyan"),
         (lon_abs_max, lat_abs_max, "Absolute Max", "black")],
        vdims=["Description", "Color"],
        crs=ccrs.PlateCarree(),
    ).opts(
        size=10, marker="x", color="Color", show_legend=True
    )
    
    # Add caption for details
    caption = (
        f"Date: {date}\n"
        f"99.9th Percentile Max: {percentile_max_value:.2f}°C | Absolute Max: {absolute_max_value:.2f}°C\n"
        f"99.9th Percentile Location: ({lat_percentile:.2f}, {lon_percentile:.2f})\n"
        f"Absolute Max Location: ({lat_abs_max:.2f}, {lon_abs_max:.2f})"
    )
    
    return hv.Div(caption) + (quadmesh * markers)

# Panel layout
layout = pn.Column(
    "# Interactive Humidex Plot for Different GWLs",
    gwl_selector,
    year_selector,
    plot_humidex,
)

# Ensure the year slider is updated dynamically
update_year_slider(gwl_selector.value)

# Save as standalone HTML file
layout.save("interactive_humidex_plot_fixed.html", embed=True)

# Serve the layout (if running in a notebook environment)
layout.servable()  # Fixed typo here


Column
    [0] Markdown(str)
    [1] Select(options=['1.0', '1.5', ...], value='4.0')
    [2] IntSlider(end=2071, name='Select Year', start=2052, value=2052)
    [3] ParamFunction(function, _pane=HoloViews)